In [1]:
import requests
import pandas as pd

### Requete et reccupération des données de fréquentation de gare


In [2]:
url = "https://ressources.data.sncf.com/api/records/1.0/search/?dataset=frequentation-gares&q=paris&sort=nom_gare&facet=nom_gare&facet=code_postal&facet=segmentation_drg"
response = requests.get(url)
data = response.json()

gares = []

for record in data["records"]:
    gare_info = {
        "nom_gare": record["fields"]["nom_gare"],
        "code_postal": record["fields"]["code_postal"],
        "frequentation_2019": record["fields"]["total_voyageurs_2019"],
        "frequentation_2020": record["fields"]["total_voyageurs_2020"],
        "frequentation_2021": record["fields"]["total_voyageurs_2021"],
    }
    gares.append(gare_info)


    print(response.status_code)

200
200
200
200
200
200
200
200


### de la DataFrame

In [3]:
df_freq_gare = pd.DataFrame(gares)

In [4]:
df_freq_gare

,nom_gare,code_postal,frequentation_2019,frequentation_2020,frequentation_2021
0,Cormeilles-en-Parisis,95240,3683266,1878474,3317416
1,Paris Austerlitz,75013,21682820,13203603,22281301
2,Paris Bercy Bourgogne - Pays d'Auvergne,75012,4318464,2498330,3347763
3,Paris Est,75010,41240098,21580625,27758686
4,Paris Gare de Lyon,75012,112644337,55824824,79986275
5,Paris Gare du Nord,75010,244759909,92236287,126622180
6,Paris Montparnasse,75015,61374035,32439307,41043477
7,Paris Saint-Lazare,75008,107340421,114468385,91502192


### Suppression de la gare Cormeilles-en-Parisis

In [5]:
df_freq_gare = df_freq_gare.drop(0)

In [6]:
df_freq_gare

,nom_gare,code_postal,frequentation_2019,frequentation_2020,frequentation_2021
1,Paris Austerlitz,75013,21682820,13203603,22281301
2,Paris Bercy Bourgogne - Pays d'Auvergne,75012,4318464,2498330,3347763
3,Paris Est,75010,41240098,21580625,27758686
4,Paris Gare de Lyon,75012,112644337,55824824,79986275
5,Paris Gare du Nord,75010,244759909,92236287,126622180
6,Paris Montparnasse,75015,61374035,32439307,41043477
7,Paris Saint-Lazare,75008,107340421,114468385,91502192


### Creation d'une nouvelle colonne "frequantation_2022" qui prends la moyenne des 3 année précedante

In [7]:
df_freq_gare = df_freq_gare.assign(
    frequentation_2022=(df_freq_gare["frequentation_2019"] + df_freq_gare["frequentation_2020"] + df_freq_gare["frequentation_2021"]) / 3
)

### Transformation de cette nouvelle colonne en chiffres entier

In [8]:
df_freq_gare["frequentation_2022"] = df_freq_gare["frequentation_2022"].astype(int)
df_freq_gare.to_csv("frequentations_gare")

### Renomme le nom de la gare pour plus de cohérence avec les autres DataFrame et Base de donnée

In [11]:
df_freq_gare['nom_gare'] = df_freq_gare['nom_gare'].replace("Paris Bercy Bourgogne - Pays d'Auvergne", "Paris Bercy")


In [15]:
coordonees = [
    (48.840609, 2.366563),  # Paris Austerlitz
    (48.840889, 2.381641),  # Paris Bercy
    (48.876576, 2.359084),  # Paris Est
    (48.844512, 2.373522),  # Paris Gare de Lyon
    (48.880931, 2.355158),  # Paris Gare du Nord
    (48.840847, 2.319543),  # Paris Montparnasse
    (48.876141, 2.326190)   # Paris Saint-Lazare
]

df_freq_gare['latitude'] = [coord[0] for coord in coordonees]
df_freq_gare['longitude'] = [coord[1] for coord in coordonees]

In [16]:
df_freq_gare

,nom_gare,code_postal,frequentation_2019,frequentation_2020,frequentation_2021,frequentation_2022,latitude,longitude
1,Paris Austerlitz,75013,21682820,13203603,22281301,19055908,48.840609,2.366563
2,Paris Bercy,75012,4318464,2498330,3347763,3388185,48.840889,2.381641
3,Paris Est,75010,41240098,21580625,27758686,30193136,48.876576,2.359084
4,Paris Gare de Lyon,75012,112644337,55824824,79986275,82818478,48.844512,2.373522
5,Paris Gare du Nord,75010,244759909,92236287,126622180,154539458,48.880931,2.355158
6,Paris Montparnasse,75015,61374035,32439307,41043477,44952273,48.840847,2.319543
7,Paris Saint-Lazare,75008,107340421,114468385,91502192,104436999,48.876141,2.326190


In [17]:
df_freq_gare.to_csv("frequentations_gare.csv", index=False)